In [3]:
%reload_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

pd.options.display.max_columns = 999
pd.options.display.max_rows = 1999
pd.options.display.max_colwidth = 200

from paths import *
from scripts import manipulate
from scripts import io

import glob

from pathlib import Path

In [75]:
df = pd.read_csv('/home/m/Downloads/COVID.csv',encoding='latin1', sep=';',nrows=100, dtype='str')

#padroniza colunas
df.columns = manipulate.normalize_cols(df.columns)

# substitui valores de fonte
map_fonte = {
    '1':'Estadual',
    '3':'Estadual',
    '4':'Estadual',
    '5':'Federal',
    '7':'Empréstimo',
}
df['fonte'] = df['fonte'].map(map_fonte)

## ajusta valores para float
value_cols = ['quantidade','valor_unitario','empenho','valor_ne','valor_nl','valor_np','valor_ob']
for col in value_cols:
    df[col] = pd.to_numeric(df[col].str.replace('.','').str.replace(',','.'), downcast='float')
    
df['valor_total'] = df['quantidade']*df['valor_unitario']

In [70]:
df.head()

,secretaria,numero_do_processo,modalidade_de_contratacao,contratada__conveniada,cpf__cnpj__cgc,descricao_processo,finalidade_item,data_de_celebracao,prazo_contratual,periodo_prazo_contratual,status_licitacao,quantidade,valor_unitario,empenho,nota_de_empenho,data_da_movimentacao,tipo_de_pagamento,numero_de_pagamento,valor_ne,valor_nl,valor_np,valor_ob,fonte,codigo_nome_fonte_detalhada,local_entrega,link_evento_licitacao_contrato,link_licitacao,link_processo
0,SECRETARIA DA EDUCACAO,43898/20,TRANSF.A INST.PRIVADAS SEM FINS LUCRATIVOS,APM DA EE AUREA ANUNCIACAO AMERICO GODOY,"5,01045E+13",PDDE PAULISTA - COVID-19,PDDE PAULISTA - PROJETOS ESPECIAIS - PROCESSO PARA REPASSE DE RECURSOS àS APM AQUISIçãO DE INSUMOS PARA COMBATE/PREVENçãO à COVID-19,NaN,NaN,NaN,NaN,0.0,0.00,4.767000e+03,2020NE03280,01/10/2020,OB,2020OB60933,0.000000,0.000000,0.0,4767.0,Federal,005003002 - SALARIO EDUCACAO-CTA-PTE CONTRIB.2/3,CAMPINAS,NaN,NaN,NaN
1,SECRETARIA DE DESENVOLVIMENTO SOCIAL,1728592/19,TRANSF.A INST.PRIVADAS SEM FINS LUCRATIVOS,AS.CASA DE APOIO N.SENHORA APARECIDA CARA,"5,67593E+12",FORNECIMENTO DE REFEIÇÕES P/ SUBVENÇÃO SOCIAL,"TERMO DE COLABORAÇÃO QUE ENTRE SI CELEBRAM O ESTADO DE SÃO PAULO, POR INTERMÉDIO DA SECRETARIA DE DESENVOLVIMENTO SOCIAL, E A ASSOCIAÇÃO CASA DE APOIO ROMEIROS DE NOSSA SENHORA APARECIDA, OBJETIVA...",NaN,NaN,NaN,NaN,0.0,0.00,1.357056e+06,2020NE00048,12/02/2020,NL,2020NL00741,0.000000,30374.960938,0.0,0.0,Estadual,001016006 - ADICIONAL ICMS-FDO EST DE COMBATE A POBREZA,LIMEIRA,NaN,NaN,NaN
2,SECRETARIA DE LOGISTICA E TRANSPORTES,1470345/20,APLICACOES DIRETAS,NANO4YOU PERFOMANCE IMPORTAÇAO E DISTRIBUIÇAO DE P,"2,66064E+13",MATERIAIS,MATERIAIS,NaN,NaN,NaN,NaN,0.0,0.00,1.024650e+03,2020NE00113,28/08/2020,NL,2020NL01001,0.000000,1024.650024,0.0,0.0,Estadual,004001133 - REC.PROP.ADM.IND-COVID19,"AVENIDA DA SAUDADE,37",NaN,NaN,NaN
3,SECRETARIA DE LOGISTICA E TRANSPORTES,1470345/20,APLICACOES DIRETAS,JR COMERCIO DE ARTIGOS EM GERAL EIRELI,"3,08508E+13",MATERIAIS,"HIGIENIZADOR,GEL,NEUTRO,ALCOOL ETILICO 70%,BACTERICIDA/ANTISSEPTICO",NaN,NaN,NaN,NaN,15.0,21.92,3.288000e+02,2020NE00114,13/08/2020,NE,2020NE00114,328.799988,0.000000,0.0,0.0,Estadual,004001133 - REC.PROP.ADM.IND-COVID19,"AVENIDA DA SAUDADE,37",NaN,NaN,NaN
4,SECRETARIA DE DESENVOLVIMENTO SOCIAL,1655101/19,TRANSF.A INST.PRIVADAS SEM FINS LUCRATIVOS,INSTITUTO PROPAV,"1,30946E+12",FORNEC. DE REFEIÇÕES B.P. DE BRASILÂNDIA,"TERMO DE COLABORAÇÃO QUE ENTRE SI CELEBRAM O ESTADO DE SÃO PAULO, POR INTERMÉDIO DA SECRETARIA DE DESENVOLVIMENTO SOCIAL, E O INSTITUTO PROPAV , OBJETIVANDO A DISPONIBILIZAÇÃO DE ALIMENTAÇÃO DE QU...",NaN,NaN,NaN,NaN,0.0,0.00,9.581911e+05,2020NE00030,12/02/2020,NL,2020NL00711,0.000000,27148.000000,0.0,0.0,Estadual,001016006 - ADICIONAL ICMS-FDO EST DE COMBATE A POBREZA,SAO PAULO,NaN,NaN,NaN


In [77]:
df[['valor_total','empenho','valor_ne','valor_nl','valor_np','valor_ob']].sum()

valor_total    1.404788e+07
empenho        2.647332e+08
valor_ne       1.405652e+07
valor_nl       6.558370e+05
valor_np       9.861102e+05
valor_ob       1.404736e+06
dtype: float32

In [76]:
group_cols = ['codigo_nome_fonte_detalhada','fonte']
value_cols = ['valor_total','empenho','valor_ne','valor_nl','valor_np','valor_ob']

df[group_cols + value_cols].groupby(group_cols, as_index=False).sum()

,codigo_nome_fonte_detalhada,fonte,valor_total,empenho,valor_ne,valor_nl,valor_np,valor_ob
0,001001001 - TESOURO-GERAL,Estadual,1.494000e+03,1.494000e+03,1.494000e+03,0.000000,0.000000,0.000000
1,001001133 - REC.TESOSURO-COVID19,Estadual,4.595260e+06,2.658000e+06,4.595260e+06,0.000000,250000.000000,281410.000000
2,001001141 - TESOURO - FUNDES,Estadual,5.216329e+05,1.620947e+07,5.302634e+05,177553.593750,46063.203125,111463.203125
3,001002007 - FUNDO DESENV.EDUCACAO BASICA-FUNDEB,Estadual,8.888527e+06,2.179141e+08,8.888526e+06,12075.259766,0.000000,4111.950195
4,001016006 - ADICIONAL ICMS-FDO EST DE COMBATE A POBREZA,Estadual,0.000000e+00,2.738133e+07,0.000000e+00,308793.343750,0.000000,230670.953125
5,003001051 - RENDIMENTOS DA QESE - CTA.-PTE.-CONTRIB.2/3,Estadual,1.372700e+04,2.452900e+04,1.372700e+04,2272.000000,0.000000,300.000000
6,003001922 - RECURSOS RESOLUCAO ALESP 922/2020,Estadual,0.000000e+00,3.326311e+05,0.000000e+00,0.000000,0.000000,57450.062500
7,004001133 - REC.PROP.ADM.IND-COVID19,Estadual,2.724495e+04,2.826960e+04,2.724495e+04,1024.650024,0.000000,0.000000
8,005001133 - REC.TRANSF.FEDERAIS-COVID19,Federal,0.000000e+00,1.541181e+05,0.000000e+00,154118.125000,0.000000,0.000000
9,005003002 - SALARIO EDUCACAO-CTA-PTE CONTRIB.2/3,Federal,0.000000e+00,2.928300e+04,0.000000e+00,0.000000,0.000000,29283.000000
